<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/Partition_the_data_by_spent_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


run the following code in big query and generate joint_date
```
SELECT 
  value,
  FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
  FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  EXTRACT(ISOYEAR FROM block_timestamp) AS year
FROM `crypto-291811.UTXO.joint_all`
```



Create Partition Table
https://fivetran.com/docs/destinations/bigquery/partition-table

run the following to create dateset partition by the spent year, do the same for all block years until 2020


```
SELECT 
  value,
  block_date,
  spent_block_date
FROM `crypto-291811.UTXO.joint_date` 
WHERE spent_year<=2010
```




In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'crypto-291811'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

Now, we can first deal with the data that has creation year in 2009 or 2010, Note that the data is smaller than 1GB so that we can import directly to colab. 
After we deal with the data, we need to delete all data that has been spent up to 2010. 

In [ ]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2010")
table = client.get_table(table_ref)

df_2010 = client.list_rows(table).to_dataframe()
df_2010.head()

In [11]:
df_2010.tail()

,value,block_date,spent_block_date
268344,5000000000,2010-10-11,2020-10-11
268345,5000000000,2010-10-13,2020-10-11
268346,5000000000,2010-11-10,2020-10-11
268347,5000000000,2010-11-09,2020-10-14
268348,5000000000,2010-09-23,2020-10-17


In [ ]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

In [13]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [15]:
df_2010.to_csv('/content/drive/My Drive/UTXO/joint_2010.csv')

In [16]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2011")
table = client.get_table(table_ref)

df_2011 = client.list_rows(table).to_dataframe()
df_2011.head()

,value,block_date,spent_block_date
0,3000000000,2011-11-03,2011-11-07
1,18,2011-09-26,None
2,21000000,2011-05-20,2011-05-21
3,201000000,2011-09-25,2012-01-05
4,398000000,2011-07-05,2011-07-06


In [17]:
df_2011.to_csv('/content/drive/My Drive/UTXO/joint_2011.csv')

In [19]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2012")
table = client.get_table(table_ref)

df_2012 = client.list_rows(table).to_dataframe()
df_2012.head()

,value,block_date,spent_block_date
0,1,2012-08-15,2012-08-15
1,400000,2012-10-25,2012-10-25
2,1000000,2012-12-09,2012-12-09
3,4714850000,2012-11-22,2012-11-22
4,1350000,2012-01-25,2012-01-25


In [20]:
df_2012.to_csv('/content/drive/My Drive/UTXO/joint_2012.csv')